In [119]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from pyspark.sql import SparkSession, udf
from pyspark.sql.functions import to_timestamp, datediff
from pyspark.sql.types import IntegerType, FloatType, DoubleType, BooleanType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, Binarizer#, OneHotEncoderEstimator, StringIndexer
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [2]:
#spark1 = SparkSession.builder.getOrCreate()

spark = SparkSession.builder.appName('gh-churn').getOrCreate()
first_period = spark.read.csv('events_data/events_2016_01_01_2016_06_01.csv', header = True, inferSchema = True)
first_period.printSchema()

root
 |-- actor: string (nullable = true)
 |-- event_count: integer (nullable = true)
 |-- last_event: string (nullable = true)
 |-- first_event: string (nullable = true)
 |-- CheckRunEvent_count: integer (nullable = true)
 |-- CheckSuiteEvent_count: integer (nullable = true)
 |-- CommitCommentEvent_count: integer (nullable = true)
 |-- ContentReferenceEvent_count: integer (nullable = true)
 |-- CreateEvent_count: integer (nullable = true)
 |-- DeleteEvent_count: integer (nullable = true)
 |-- DeploymentEvent_count: integer (nullable = true)
 |-- DeploymentStatusEvent_count: integer (nullable = true)
 |-- DownloadEvent_count: integer (nullable = true)
 |-- FollowEvent_count: integer (nullable = true)
 |-- ForkEvent_count: integer (nullable = true)
 |-- ForkApplyEvent_count: integer (nullable = true)
 |-- asGitHubAppAuthorizationEvent_count: integer (nullable = true)
 |-- GistEvent_count: integer (nullable = true)
 |-- GollumEvent_count: integer (nullable = true)
 |-- InstallationEvent_

In [3]:
pd.DataFrame(first_period.take(5), columns=first_period.columns)

,actor,event_count,last_event,first_event,CheckRunEvent_count,CheckSuiteEvent_count,CommitCommentEvent_count,ContentReferenceEvent_count,CreateEvent_count,DeleteEvent_count,...,PushEvent_count,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count
0,luceiro,1,2016-02-29 16:42:12 UTC,2016-02-29 16:42:12 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,sweilemj,1,2016-02-24 21:37:39 UTC,2016-02-24 21:37:39 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,JimHallett,1,2016-03-03 05:03:29 UTC,2016-03-03 05:03:29 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SecurNET,1,2016-03-03 22:20:06 UTC,2016-03-03 22:20:06 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,mruthvend2l,1,2016-02-23 21:12:11 UTC,2016-02-23 21:12:11 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
first_period.count()

3699852

In [5]:
second_period = spark.read.csv('events_data/events_2016_06_02_2016_11_01_01.csv', header = True, inferSchema = True)
pd.DataFrame(second_period.take(5), columns=second_period.columns)

,actor,event_count,last_event,first_event,CheckRunEvent_count,CheckSuiteEvent_count,CommitCommentEvent_count,ContentReferenceEvent_count,CreateEvent_count,DeleteEvent_count,...,PushEvent_count,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count
0,tcastellanos,256,2016-10-14 22:10:05 UTC,2016-06-02 16:38:40 UTC,0,0,0,0,0,0,...,256,0,0,0,0,0,0,0,0,0
1,xamarcade,256,2016-10-28 20:09:36 UTC,2016-06-04 16:04:19 UTC,0,0,0,0,0,0,...,256,0,0,0,0,0,0,0,0,0
2,sophron,256,2016-11-01 21:35:12 UTC,2016-06-09 18:07:54 UTC,0,0,0,0,0,0,...,40,0,0,0,0,0,0,0,0,1
3,hyman720,1,2016-06-08 15:40:36 UTC,2016-06-08 15:40:36 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,gmleahy57,1,2016-06-06 12:54:00 UTC,2016-06-06 12:54:00 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
users = spark.read.csv('all_users.csv', header=True, inferSchema=True)
users = users.drop('event_count').drop('last_event').drop('first_event').drop('_c0')
pd.DataFrame(users.take(5), columns=users.columns)

,login,followers_count,following_count,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,time_between_first_last_event,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23
0,06Birch,0,0,None,None,2016-02-08 11:19:54+00:00,0,0,None,2016-02-08 11:19:54+00:00,0 days 00:19:06.000000000,None,None,None,None,None,None,None,None,None
1,06peng,78,27,http://06peng.com,None,2013-07-25 13:43:38+00:00,1,0,True,2019-01-08 01:55:29+00:00,147 days 07:31:20.000000000,None,None,None,None,None,None,None,None,None
2,0p3ns0urc3,0,0,None,None,2016-04-13 22:30:01+00:00,2,0,None,2016-04-13 22:30:01+00:00,2 days 17:17:36.000000000,None,None,None,None,None,None,None,None,None
3,100341976,0,0,None,None,2016-06-01 19:30:21+00:00,0,0,None,2016-06-01 20:31:02+00:00,0 days 01:16:16.000000000,None,None,None,None,None,None,None,None,None
4,100amps,0,0,None,None,2016-03-24 05:53:08+00:00,0,0,None,2016-05-28 03:05:32+00:00,0 days 00:16:47.000000000,None,None,None,None,None,None,None,None,None


In [167]:
churn_data = users.join(first_period, users['login'] == first_period['actor'], how='left')
pd.DataFrame(churn_data.take(5), columns=churn_data.columns)

,login,followers_count,following_count,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,...,PushEvent_count,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count
0,magnusfriis,0,0,None,None,2015-10-19 13:32:59+00:00,0,0,None,2018-05-01 20:27:16+00:00,...,59,0,0,0,0,0,0,0,0,0
1,100341976,0,0,None,None,2016-06-01 19:30:21+00:00,0,0,None,2016-06-01 20:31:02+00:00,...,0,0,0,0,0,0,0,0,0,0
2,mahabub0,0,0,None,None,2016-02-15 01:47:02+00:00,1,0,None,2016-02-15 01:47:02+00:00,...,0,0,0,0,0,0,0,0,0,0
3,06peng,78,27,http://06peng.com,None,2013-07-25 13:43:38+00:00,1,0,True,2019-01-08 01:55:29+00:00,...,0,0,0,0,0,0,0,0,0,42
4,colton5007,2,0,None,None,2013-11-30 02:35:52+00:00,9,0,True,2019-01-16 01:01:25+00:00,...,100,0,0,0,0,0,0,0,0,0


In [168]:
second_period_event_count = second_period.selectExpr("actor as login",
                                                     "event_count as second_period_event_count")
second_period_event_count.describe().show()

+-------+--------------------+-------------------------+
|summary|               login|second_period_event_count|
+-------+--------------------+-------------------------+
|  count|             3947787|                  3947788|
|   mean|            Infinity|       33.778560550870516|
| stddev|                 NaN|       1701.2997291277995|
|    min|                   0|                        0|
|    max|zzzzzzzzzzzzzzzzz...|                  2713263|
+-------+--------------------+-------------------------+



In [169]:
churn_data = churn_data.join(second_period_event_count, on='login', how='left')

In [170]:
churn_data = churn_data.withColumn("first_event", to_timestamp(churn_data.first_event))
churn_data = churn_data.withColumn("last_event", to_timestamp(churn_data.last_event))
churn_data = churn_data.withColumn("created_at", to_timestamp(churn_data.created_at))
churn_data = churn_data.withColumn("updated_at", to_timestamp(churn_data.updated_at))

In [171]:
churn_data = churn_data.withColumn("time_between_first_last_event", 
                                   datediff(churn_data.last_event, churn_data.first_event))

In [172]:
churn_data = churn_data.withColumn("public_repos_count", churn_data.public_repos_count.cast(IntegerType()))
churn_data = churn_data.withColumn("public_gists_count", churn_data.public_gists_count.cast(IntegerType()))

In [173]:
churn_data = churn_data.fillna(0, subset='second_period_event_count')

In [174]:
f_udf=udf.UserDefinedFunction(lambda x: 1 if x is not None else 0, IntegerType())

churn_data = churn_data.withColumn("blog", f_udf(churn_data.blog))
churn_data = churn_data.withColumn("company", f_udf(churn_data.company))
churn_data = churn_data.withColumn("hireable", f_udf(churn_data.hireable))


In [175]:
pd.DataFrame(churn_data.take(5), columns=churn_data.columns).transpose()

,0,1,2,3,4
login,0loky0,13246864965,1xPdd,2582056905xy,277542952
followers_count,0,0,0,2,1
following_count,1,0,0,6,28
blog,0,0,0,0,0
company,0,0,0,0,0
created_at,2011-05-24 16:15:25,2016-04-14 06:04:50,2014-07-10 23:45:56,2014-04-24 10:51:26,2016-03-15 23:17:58
public_repos_count,7,3,0,35,1
public_gists_count,0,0,0,0,0
hireable,1,0,0,0,0
updated_at,2019-01-09 10:03:59,2018-08-02 03:48:44,2018-12-15 19:55:16,2018-10-17 21:47:18,2017-08-14 15:48:16


In [176]:
churn_data = churn_data.select('login', 'followers_count', 'following_count', 'blog',
               'company', 'created_at', 'public_repos_count', 'public_gists_count',
               'hireable',
               #'updated_at', 'time_between_first_last_event', 'last_event', 'first_event',
               #'event_count', 
               'second_period_event_count',
               'CommitCommentEvent_count', 'CreateEvent_count', 'DeleteEvent_count', 
               'ForkEvent_count', 'GollumEvent_count', 'IssueCommentEvent_count',
               'IssuesEvent_count', 'MemberEvent_count', 'PublicEvent_count', 
               'PullRequestEvent_count', 'PullRequestReviewCommentEvent_count',
               'PushEvent_count', 'ReleaseEvent_count', 'WatchEvent_count')
cols = churn_data.columns
churn_data.printSchema()

root
 |-- login: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- following_count: integer (nullable = true)
 |-- blog: integer (nullable = true)
 |-- company: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- public_repos_count: integer (nullable = true)
 |-- public_gists_count: integer (nullable = true)
 |-- hireable: integer (nullable = true)
 |-- second_period_event_count: integer (nullable = true)
 |-- CommitCommentEvent_count: integer (nullable = true)
 |-- CreateEvent_count: integer (nullable = true)
 |-- DeleteEvent_count: integer (nullable = true)
 |-- ForkEvent_count: integer (nullable = true)
 |-- GollumEvent_count: integer (nullable = true)
 |-- IssueCommentEvent_count: integer (nullable = true)
 |-- IssuesEvent_count: integer (nullable = true)
 |-- MemberEvent_count: integer (nullable = true)
 |-- PublicEvent_count: integer (nullable = true)
 |-- PullRequestEvent_count: integer (nullable = true)
 |-- PullRequestReviewC